In [ ]:
import pandas as pd

def read_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print("An error occurred:", e)
        return None
    

In [ ]:
data= read_csv_to_dataframe("/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Data/102_Data.csv")
path_smote="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote"
path_strat="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/strat"
path_smote_strat="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote_strat"

In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

def tune_mlp_hyperparameters(X_train, y_train, X_test, y_test):
    param_dist = {
        'hidden_layer_sizes': [(10,), (10, 10), (50,), (50, 50), (100,), (100, 100), (200,), (200, 200)],
        'activation': ['relu', 'tanh', 'logistic', 'identity'],
        'solver': ['adam', 'lbfgs', 'sgd'],
        'alpha': (1e-5, 1e-1, 'log-uniform'),
        'learning_rate': ['constant', 'invscaling', 'adaptive'],
        'early_stopping': [True, False],
        'max_iter': range(100, 1000,100), 
    }

    mlp_classifier = MLPClassifier()

    scaler = StandardScaler()


    X_train_scaled = scaler.fit_transform(X_train)

    X_test_scaled = scaler.transform(X_test)

    cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    random_search = RandomizedSearchCV(
        mlp_classifier, param_distributions=param_dist, n_iter=50, cv=cv_strategy, scoring='accuracy', random_state=42
    )
    
    random_search.fit(X_train, y_train)

    results_df = pd.DataFrame(random_search.cv_results_)

    results_df.to_csv('hyperparameter_tuning_results_random_mlp.csv', index=False)

    print(f"Best Hyperparameters: {random_search.best_params_}")
    print(f"Best Accuracy: {random_search.best_score_}")

    best_params_values = {key: np.array(value).item() if isinstance(value, np.ndarray) else value for key, value in random_search.best_params_.items()}

    y_train_pred = random_search.best_estimator_.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    print(f"Train Accuracy: {train_accuracy}")

    y_test_pred = random_search.best_estimator_.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy: {test_accuracy}")
    return random_search.best_params_

# Assuming X_train, y_train, X_test, and y_test are available
params=tune_mlp_hyperparameters(X_train, y_train, X_test, y_test)


In [ ]:
params

In [ ]:
mlp=MLPClassifier(**params)


In [ ]:
from Utility_model_training import evaluate_classifier_with_stratified_kfold, evaluate_classifier_with_kfold_smote, evaluate_classifier_with_stratified_smote
evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, mlp, num_folds=10,save_path=path_smote_strat,model_name="mlp_classifier_smote_stratified")
evaluate_classifier_with_stratified_kfold(X_train, y_train, X_test, y_test, mlp, num_folds=10,save_path=path_strat,model_name="mlp_classifier_stratified")
evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, mlp, num_folds=10,save_path=path_smote,model_name="mlp_classifier_smote")

In [ ]:
path_smote_strat_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote_strat/ROC_Curves"
path_strat_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/strat/ROC_Curves"
path_smote_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote/ROC_Curves"
from Utility_ROC_Curves import plot_roc_curves_multiclass_smote_strat,plot_roc_curves_multiclass_smote, plot_roc_curves_multiclass_strat
plot_roc_curves_multiclass_smote_strat(mlp, X, y, n_splits=10, save_folder=path_smote_strat_ROC, model_name="mlp_KFold_SMOTE_strat")
plot_roc_curves_multiclass_strat(mlp, X, y, n_splits=10, save_folder=path_strat_ROC, model_name="mlp_KFold_Strat")
plot_roc_curves_multiclass_smote(mlp, X, y, n_splits=10, save_folder=path_smote_ROC, model_name="mlp_KFold_SMOTE")
